<a href="https://colab.research.google.com/github/benmsanderson/energybalance/blob/main/read_cmip5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4xCO2 read data


CMIP6

In [ ]:
readdata=1
authdrive=1

Desired interpolation grid

Desired experiments

In [ ]:
flds=['rsut','rlut','tas','rsdt']
expts=['abrupt-4xCO2','piControl']
calstrt=[True,True,True]
dbe=['CMIP5','CMIP5','CMIP5']

Install some stuff

In [ ]:
!pip uninstall -y albumentations >/dev/null
!pip install --upgrade xarray zarr gcsfs cftime pydsm nc-time-axis imgaug matplotlib==3.1.3 progress eofs cartopy netcdf4 >/dev/null

Import stuff

In [ ]:
from matplotlib import pyplot as plt
from netCDF4 import num2date
import numpy as np
import pydsm.relab as relab
import numpy.matlib
import pandas as pd
import xarray as xr
import zarr
import gcsfs
import pickle
import cftime
import cartopy.crs as ccrs
import dask as da
from eofs.xarray import Eof
from sys import getsizeof
from IPython.display import HTML, display
import time
import glob

xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

#code for pretty progress bars
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))


runtime has 27.3 gigabytes of available RAM



In [ ]:
lon_out=np.arange(1,359,2)
lat_out=np.arange(-89,89,2)
lons_sub, lats_sub = np.meshgrid(lon_out,lat_out)

Activate Google Drive to store arrays

In [ ]:
if authdrive:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
plt.rcParams['figure.figsize'] = 12, 6

## Browse Catalog

The data catatalog is stored as a CSV file. Here we read it with Pandas.

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/pangeo-cmip5.csv', low_memory=False)

Variables and experiments in database

In [ ]:
df.activity_id.unique()

array(['CMIP5'], dtype=object)

In [ ]:
vars=df.variable_id.unique()
vars.sort()
expts_full=df.experiment_id.unique()
expts_full.sort()
expts_full

array(['1pctCO2', 'abrupt-4xCO2', 'historical', 'piControl', 'rcp26',
       'rcp85'], dtype=object)

Make dataframe for each experiment type and each field

In [ ]:
df_all1=[]
for i, row in enumerate(expts):
  df_ta1=[]
  for j,fld in enumerate(flds):
    tmp = df.query("activity_id=='"+dbe[i]+"' & table_id == 'Amon' & variable_id == '"+fld+"' & experiment_id == '"+expts[i]+"'")
    df_ta1.append(tmp)
  df_all1.append(df_ta1)

Isolate unique models which have completed 4xco2

In [ ]:
mdls1=df_all1[0][0].source_id.unique()
mdls1.sort()
mdls1

array(['ACCESS1-0', 'ACCESS1-3', 'BNU-ESM', 'CCSM4', 'CESM1-CAM5-1-FV2',
       'CNRM-CM5', 'CNRM-CM5-2', 'CSIRO-Mk3-6-0', 'CanESM2', 'FGOALS-g2',
       'FGOALS-s2', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'GISS-E2-H',
       'GISS-E2-R', 'HadGEM2-ES', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR',
       'IPSL-CM5B-LR', 'MIROC-ESM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR',
       'MPI-ESM-P', 'MRI-CGCM3', 'NorESM1-M', 'bcc-csm1-1',
       'bcc-csm1-1-m', 'inmcm4'], dtype=object)

Make some empty dataframes to store concise list

In [ ]:
cnames=df_all1[0][0].columns
df_all=[]
for i, exp in enumerate(expts):
  tmp=[]
  for j,fld  in enumerate(flds):
    tmp.append(pd.DataFrame(columns=cnames))
  df_all.append(tmp)

Now get 1 ensemble member for each model, if it exists, for each experiment.  Only add to dataframe df_ta if we have a full set of experiments

In [ ]:
mdls=[]
n=0
for j, mdl in enumerate(mdls1):
    tmpdf=[]
    nruns=[]
    for i, ext in enumerate(expts):
        #find first variable for expt/model
        for j, fld in enumerate(flds):
          tmp=df_all1[i][j].query("source_id=='"+mdl+"'")
          nruns.append(tmp.shape[0])
    #is there at least 1 run per experiment,with all fields?
    if min(nruns)>=1:
      #point to the entry for 1st run, first variable for each expt
      for i, ext in enumerate(expts):
        mmb=df_all1[i][0]['member_id'].values[0]
        for j, fld in enumerate(flds):
            #tt = df_all1[i][j].query("source_id=='"+mdl+"' & table_id == 'Amon' & member_id == '"+mmb+"'")
            tt = df_all1[i][j].query("source_id=='"+mdl+"' & table_id == 'Amon'")
            df_all[i][j].loc[n]=tt.values[0]

      #add model to final list
      mdls.append(mdl)
      n=n+1
    
    

In [ ]:
df_all1[i][j].query("source_id=='"+mdl+"'")

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
861,CMIP5,INM,inmcm4,piControl,r1i1p1f1,Amon,rsdt,gn,gs://cmip6/CMIP5/INM/inmcm4/piControl/r1i1p1f1...,NaN


In [ ]:
mdls

['ACCESS1-0',
 'ACCESS1-3',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'CanESM2',
 'FGOALS-g2',
 'FGOALS-s2',
 'GFDL-CM3',
 'GFDL-ESM2G',
 'GFDL-ESM2M',
 'GISS-E2-H',
 'GISS-E2-R',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'IPSL-CM5A-MR',
 'IPSL-CM5B-LR',
 'MIROC-ESM',
 'MIROC5',
 'MPI-ESM-LR',
 'MPI-ESM-P',
 'NorESM1-M',
 'bcc-csm1-1',
 'bcc-csm1-1-m',
 'inmcm4']

In [ ]:
pickle.dump(mdls, open( "/content/drive/MyDrive/colab_4xco2/mdls_cmip5.pkl", "wb" ) )


In [ ]:
df_all[0][2]

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
0,CMIP5,CSIRO-BOM,ACCESS1-0,abrupt-4xCO2,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP5/CSIRO-BOM/ACCESS1-0/abrupt-4x...,NaN
1,CMIP5,CSIRO-BOM,ACCESS1-3,abrupt-4xCO2,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP5/CSIRO-BOM/ACCESS1-3/abrupt-4x...,NaN
2,CMIP5,NCAR,CCSM4,abrupt-4xCO2,r2i1p1f1,Amon,tas,gn,gs://cmip6/CMIP5/NCAR/CCSM4/abrupt-4xCO2/r2i1p...,NaN
3,CMIP5,CNRM-CERFACS,CNRM-CM5,abrupt-4xCO2,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP5/CNRM-CERFACS/CNRM-CM5/abrupt-...,NaN
4,CMIP5,CSIRO-QCCCE,CSIRO-Mk3-6-0,abrupt-4xCO2,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP5/CSIRO-QCCCE/CSIRO-Mk3-6-0/abr...,NaN
5,CMIP5,CCCMA,CanESM2,abrupt-4xCO2,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP5/CCCMA/CanESM2/abrupt-4xCO2/r1...,NaN
6,CMIP5,LASG-CESS,FGOALS-g2,abrupt-4xCO2,r2i1p1f1,Amon,tas,gn,gs://cmip6/CMIP5/LASG-CESS/FGOALS-g2/abrupt-4x...,NaN
7,CMIP5,LASG-IAP,FGOALS-s2,abrupt-4xCO2,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP5/LASG-IAP/FGOALS-s2/abrupt-4xC...,NaN
8,CMIP5,NOAA-GFDL,GFDL-CM3,abrupt-4xCO2,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP5/NOAA-GFDL/GFDL-CM3/abrupt-4xC...,NaN
9,CMIP5,NOAA-GFDL,GFDL-ESM2G,abrupt-4xCO2,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP5/NOAA-GFDL/GFDL-ESM2G/abrupt-4...,NaN


## Load Data

Load Google file system


In [ ]:
# load Google cloud storage
gcs = gcsfs.GCSFileSystem(token='anon')

Loop through zstore links, use zarr to open


In [ ]:
nm=len(mdls)
nf=len(flds)
ne=len(expts)

In [ ]:
flds

['rsut', 'rlut', 'tas', 'rsdt']

In [ ]:

if readdata:
  out = display(progress(0, 1), display_id=True)
  dsall=[]
  for i,df_ta in enumerate(df_all):
    dsm=[]
    for j,df in enumerate(df_ta):
      ds=[]
      print(expts[i]+','+flds[j])
      for index, item in enumerate(df.zstore.values, start=0):
        mapper=gcs.get_mapper(item)
        ds.append(xr.open_zarr(mapper, decode_times=False))
        out.update(progress(index+j*nm+i*nm*nf, ne*nm*nf))
      dsm.append(ds)
    dsall.append(dsm)  

abrupt-4xCO2,rsut
abrupt-4xCO2,rlut
abrupt-4xCO2,tas
abrupt-4xCO2,rsdt
piControl,rsut
piControl,rlut
piControl,tas
piControl,rsdt


concatenated dataarrays for ts, global mean


In [ ]:

if readdata:
  dall=[]
  out = display(progress(0, 1), display_id=True)
  for i,ds in enumerate(dsall,start=0):
    dexp=[]
    for j,dm in enumerate(ds):
      print(expts[i]+','+flds[j])
      for index, dd in enumerate(dm, start=0):
          if 'longitude' in dd.keys():
            dd=dd.rename({'longitude': 'lon','latitude': 'lat'})
          if 'latitude' in dd.coords:
            dd=dd.drop('latitude')  
            dd=dd.drop_dims('latitude')
       
          tmp=dd[flds[j]][:4800,:,:].interp(lon=lon_out,lat=lat_out, kwargs={"fill_value": "extrapolate"})
          if calstrt[i]:
            tmp.coords['time']=pd.date_range('1850-01-01', periods=tmp['time'].values.shape[0],freq='M')
          if tmp['time'].dtype=='float64' or  tmp['time'].dtype=='int64':
            tmp.coords['time']=num2date(tmp['time'].values,tmp['time'].units)
          if 'historical' in expts[i]:
            tmpf=dsall[expts.index('ssp585')][j][index][flds[j]].interp(lon=lon_out,lat=lat_out, kwargs={"fill_value": "extrapolate"})
            if tmpf['time'].dtype=='float64' or  tmpf['time'].dtype=='int64':
              tmpf.coords['time']=num2date(tmpf['time'].values,tmpf['time'].units)
            tmp=xr.concat([tmp,tmpf],'time')
            tmp=tmp.where(tmp['time.year'] < 2021, drop=True)
          if 'ssp' in expts[i]:
            tmp=tmp.where(tmp['time.year'] > 2014, drop=True)        
          srm=tmp.groupby('time.year').mean('time')
          if index==0:
            dac=srm
          else:
            dac=xr.concat([dac,srm],'ens',coords='minimal',compat='override')
          out.update(progress(index+j*nm+i*nm*nf, ne*nm*nf))
      dexp.append(dac)
    dall.append(dexp)


abrupt-4xCO2,rsut
abrupt-4xCO2,rlut
abrupt-4xCO2,tas
abrupt-4xCO2,rsdt
piControl,rsut
piControl,rlut
piControl,tas
piControl,rsdt


In [ ]:
dd

<xarray.Dataset>
Dimensions:    (lat: 120, bnds: 2, lon: 180, time: 6000)
Coordinates:
  * lat        (lat) float64 -89.25 -87.75 -86.25 -84.75 ... 86.25 87.75 89.25
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(120, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
  * time       (time) int64 0 708 1416 2148 ... 4377060 4377792 4378524 4379256
    time_bnds  (time, bnds) float64 dask.array<chunksize=(6000, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    rsdt       (time, lat, lon) float32 dask.array<chunksize=(578, 120, 180), meta=np.ndarray>
Attributes: (12/27)
    Conventions:            CF-1.4
    branch_time:            0.0
    cmor_version:           2.0.0
    comment:                no comments
    contact:                Evgeny Volodin, volodin@inm.ras.ru,INM RAS, Gubki...
    creation_date:          2010-06-01T14:17:47Z
    ...                     ...
    realization:            1
    references:             Volodin, Diansky, Gusev 2010. Climate model INMCM...
    source:                 inmcm4 (2009)
    table_id:               Table Amon (12 May 2010) 12e195f2fdc87c907b38b38e...
    title:                  inmcm4 model output prepared for CMIP5 pre-indust...
    tracking_id:            47d7cee5-e0ce-4ee1-8346-ba7c78da9d58

Find complete runs




In [ ]:

if readdata:
  out = display(progress(0, 1), display_id=True)
  for i,d in enumerate(dall,start=0):
    tmp=xr.merge(d[:])
    tmp.to_netcdf('/content/drive/MyDrive/colab_4xco2/cmip5/'+expts[i]+'.nc')
    lat=tmp.tas.lat
    weights = np.cos(np.deg2rad(lat))
    weights.name = "weights"
    tmp_gm=tmp.mean(dim='year').weighted(weights).mean(dim='lat').mean(dim='lon')
    tmp_gm.to_netcdf('/content/drive/MyDrive/colab_4xco2/cmip5/'+expts[i]+'_gm.nc')
    out.update(progress(i,ne))


/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [ ]:
mdls

['ACCESS1-0',
 'ACCESS1-3',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'CanESM2',
 'FGOALS-g2',
 'FGOALS-s2',
 'GFDL-CM3',
 'GFDL-ESM2G',
 'GFDL-ESM2M',
 'GISS-E2-H',
 'GISS-E2-R',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'IPSL-CM5A-MR',
 'IPSL-CM5B-LR',
 'MIROC-ESM',
 'MIROC5',
 'MPI-ESM-LR',
 'MPI-ESM-P',
 'NorESM1-M',
 'bcc-csm1-1',
 'bcc-csm1-1-m',
 'inmcm4']